In [1]:
from IPython.display import clear_output
import cx_Oracle


In [8]:
import pandas
from bs4 import BeautifulSoup
import urllib.request
import csv
import pandas as pd
import numpy as np

excel_data_df = pandas.read_excel('HS Code List.xlsx',sheet_name='SH', usecols=['Code de SH', 'Description du SH'])

excel_data_df['chapitre']=excel_data_df['Code de SH'].astype(str).str.slice(0,2)
excel_data_df['position']=excel_data_df['Code de SH'].astype(str).str.slice(0,4)
excel_data_df=excel_data_df.assign(section='0')
section_id=[]
section_title=[]
dico={'I':(6,6,"ANIMAUX VIVANTS ET PRODUITS DU REGNE ANIMAL"),'II':(5,14,"PRODUITS DU REGNE VEGETAL"),
      'III':(15,15,"GRAISSES ET HUILES ANIMALES OU VEGETALES; PRODUITS DE LEUR DISSOCIATION; GRAISSES ALIMENTAIRES ELABOREES; CIRES D'ORIGINE ANIMALE OU VEGETALE")
      ,'IV':(16,25,"PRODUITS DES INDUSTRIES ALIMENTAIRES; BOISSONS, LIQUIDES ALCOOLIQUES ET VINAIGRES; TABACS ET SUCCEDANES DE TABAC FABRIQUES"),
      'V':(25,27,"PRODUITS MINERAUX"),'VI':(28,38,"PRODUITS DES INDUSTRIES CHIMIQUES OU DES INDUSTRIES CONNEXES"),
      'VII':(39,40,"MATIERES PLASTIQUES ET OUVRAGES EN CES MATIERES; CAOUTCHOUC ET OUVRAGES EN CAOUTCHOUC"),
      'VIII':(41,43,"PEAUX, CUIRS, PELLETERIES ET OUVRAGES EN CES MATIERES; ARTICLES DE BOURRELLERIE OU DE SELLERIE; ARTICLES DE VOYAGE, SACS A MAIN ET CONTENANT SIMILAIRES; OUVRAGES EN BOYAUX"),
      'IX':(44,46,"BOIS, CHARBON DE BOIS ET OUVRAGES EN BOIS; LIEGE ET OUVRAGES EN LIEGE; OUVRAGES DE SPARTERIE OU DE VANNERIE"),'X':(47,49,"PATES DE BOIS OU D'AUTRES MATIERES FIBREUSES CELLULOSIQUES; PAPIER OU CARTON A RECYCLER (DECHETS ET REBUTS); PAPIER ET SES APPLICATIONS"),
      'XI':(50,63,"MATIERES TEXTILES ET OUVRAGES EN CES MATIERES"),'XII':(64,67,"CHAUSSURES, COIFFURES, PARAPLUIES, PARASOLS, CANNES, FOUETS, CRAVACHES ET LEURS PARTIES; PLUMES APPRETEES ET ARTICLES EN PLUMES; FLEURS ARTIFICIELLES; OUVRAGES EN CHEVEUX"),
      'XIII':(68,70,"OUVRAGES EN PIERRES, PLATRE, CIMENT, AMIANTE, MICA OU MATIERES ANALOGUES; PRODUITS CERAMIQUES; VERRE ET OUVRAGES EN VERRE"),'XIV':(71,71,"PERLES FINES OU DE CULTURE, PIERRES GEMMES OU SIMILAIRES, METAUX PRECIEUX, PLAQUES OU DOUBLES DE METAUX PRECIEUX ET OUVRAGES EN CES MATIERES; BIJOUTERIE DE FANTAISIE; MONNAIES"),'XV':(72,83,"METAUX COMMUNS ET OUVRAGES EN CES METAUX"),
      'XVI':(84,85,"MACHINES ET APPAREILS, MATERIEL ELECTRIQUE ET LEURS PARTIES; APPAREILS D'ENREGISTREMENT OU DE REPRODUCTION DU SON, APPAREILS D'ENREGISTREMENT OU DE REPRODUCTION DES IMAGES ET DU SON EN TELEVISION, ET PARTIES ET ACCESSOIRES DE CES APPAREILS"),'XVII':(86,89,"MATERIEL DE TRANSPORT"),
      'XVIII':(90,92,"INSTRUMENTS ET APPAREILS D'OPTIQUE, DE PHOTOGRAPHIE OU DE CINEMATOGRAPHIE, DE MESURE, DE CONTROLE OU DE PRECISION; INSTRUMENTS ET APPAREILS MEDICO-CHIRURGICAUX; HORLOGERIE; INSTRUMENTS DE MUSIQUE; PARTIES ET ACCESSOIRES DE CES INSTRUMENTS OU APPAREILS"),
      'XIX':(93,93,"ARMES, MUNITIONS ET LEURS PARTIES ET ACCESSOIRES"),'XX':(94,96,"MARCHANDISES ET PRODUITS DIVERS"),'XXI':(97,97,"OBJETS D'ART, DE COLLECTION OU D'ANTIQUITE")};
for key,val in dico.items():
    excel_data_df.loc[(val[0]<=excel_data_df['chapitre'].astype(float)) &  (excel_data_df['chapitre'].astype(float)<=val[1]), 'section']=key
    section_id.append(key)
    section_title.append(val[2])
    
#*********************************************************************************************************
                                    #extraction des sections
#*******************************************************************************************************

de={"numero":section_id,"titre":section_title}
df_section = pd.DataFrame (de, columns = ['numero','titre'])


#*********************************************************************************************************
                                    #extraction des chapitres
#*******************************************************************************************************


urlpage = 'http://www.wcoomd.org/fr/topics/nomenclature/instrument-and-tools/hs-nomenclature-2017-edition/hs-nomenclature-2017-edition.aspx'
page = urllib.request.urlopen(urlpage)
soup = BeautifulSoup(page, 'html.parser')
table=soup.find('table', attrs={'border': '0' ,"style":"width: 100%; height: 100%;"})
num=[]
chap=[]

#table = soup.find('table')
for row in table.findAll('td', attrs={"style":"width: 7%;"}):
    a = row.text.replace('\n', ' ').replace('\r', '')
    a = row.text.replace('\n', ' ').replace('\r', '')

    num.append((a))
    
for row in table.findAll('td', attrs={"style":"width: 73%;"}):
    a = row.text.replace('\n', ' ').replace('\r', '')
    chap.append(a)
    
de={"numero":num,"titre":chap}

df1 = pd.DataFrame (de, columns = ['numero'])
df1=df1.iloc[:172]

df2 = pd.DataFrame (de, columns = ['titre'])
#df1 = df1[df1.numero!='' ]
df=pandas.concat([df1, df2], axis = 1)
df['numero'] = df['numero'].str.replace('\D', '')
df['numero'].replace('', np.nan, inplace=True)
df.dropna(subset = ["numero"], inplace=True)
df = df.astype(str).apply(lambda x: x.str[:2000])
excel_data_df = excel_data_df.astype(str).apply(lambda x: x.str[:2000])



writer=pd.ExcelWriter('Position.xlsx')  
excel_data_df.to_excel(writer, sheet_name='position',index=False)
writer.save()
writer=pd.ExcelWriter('chapitre.xlsx')  

df.to_excel(writer, sheet_name='Chapitre',index=False)
writer.save()
writer=pd.ExcelWriter('section.xlsx')  

df_section.to_excel(writer, sheet_name='section',index=False)

writer.save()

In [5]:
class OraExec(object):
    _db_connection = None
    _db_cur = None

    def __init__(self):
        self._db_connection = cx_Oracle.connect('webguce/webguce@192.168.56.3:1521/GUCE', encoding="UTF-8",nencoding="UTF-8")
        self._db_cur = self._db_connection.cursor()
    #con=OraExec()
    def insertPositionTarifaire(self):
        

        sql='insert into webguce.tarif_position values(:1,:2,:3,:4,:5)'
        df_list = excel_data_df.values.tolist()
        n = 0
        for i in excel_data_df.iterrows():
            self._db_cur.execute(sql,df_list[n])
            n += 1

        self._db_connection.commit()

        return self._db_cur
    
    
    def insertchapter(self):
        sql='insert into chapter (title,num_)values(:title,:num_)'
        df_list = df.values.tolist()
        n = 0
        for i in df.iterrows():
            print(str(df_list[n][0]))

            self._db_cur.execute(sql,[str(df_list[n][1]),str(df_list[n][0])])

            n += 1


    
    def insertsection(self):
        sql='insert into webguce.section_ values(:1,:2)'
        df_list = df_section.values.tolist()
        n = 0
        for i in df_section.iterrows():
            n += 1

        self._db_connection.commit()

        return self._db_cur
    
   


    
con=OraExec()

    
cur=con.insertchapter()
#cur=con.insertsection()
con._db_connection.commit()
con._db_cur.close()

con._db_connection.close()

#cur=con.insertPositionTarifaire()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


DatabaseError: ORA-12899: value too large for column "WEBGUCE"."CHAPTER"."TITLE" (actual: 295, maximum: 255)

In [4]:
con._db_cur.close()

con._db_connection.close()